In [2]:
import warnings
warnings.filterwarnings("ignore")
from MT5 import *
import numpy as np
import time
import MetaTrader5 as mt5

def pair_trading(ts1_symbol, ts2_symbol, pair=1):
    mt5.initialize()

    #Import the data
    ts1_data = MT5.get_data(ts1_symbol, 3500)["close"]
    ts2_data = MT5.get_data(ts2_symbol, 3500)["close"]
    train_set = pd.concat((ts1_data, ts2_data), axis=1).dropna()
    train_set.columns = [ts1_symbol, ts2_symbol]
    
    train_set["spread"] = train_set[ts1_symbol]-train_set[ts2_symbol]

    train_set["z-score"] = (train_set["spread"] - train_set["spread"].mean()) / train_set["spread"].std()

    mean = train_set["z-score"].mean()
    std_high = train_set["z-score"].std() + mean
    std_low = train_set["z-score"].std() - mean
    
    # Yersteday
    yts1 = train_set[ts1_symbol].values[-2]
    yts2 = train_set[ts2_symbol].values[-2]
    
    #Today
    pts1 = train_set[ts1_symbol].values[-1]
    pts2 = train_set[ts2_symbol].values[-1]
    
    
    # Today data
    spread = pts1 - pts2
    zscore = (spread-train_set["spread"].mean())/train_set["spread"].std()
    
    # Yersteday
    yspread = yts1 - yts2
    yzscore = (yspread-train_set["spread"].mean())/train_set["spread"].std()
    
    
    # TS1
    short_ts1 = False
    long_ts1 = False
    
    if zscore < std_high and yzscore > std_high:
        short_ts1 = True
        long_ts1 = False
    
        
    elif zscore > std_low and yzscore < std_low:
        short_ts1 = False
        long_ts1 = True
        
    else:
        pass
        
    #TS2
    short_ts2 = False
    long_ts2 = False
    
    if zscore < std_high and yzscore > std_high:
        short_ts2 = False
        long_ts2 = True
  
    elif zscore > std_low and yzscore < std_low:
        short_ts2 = True
        long_ts2 = False
   
    else:
        pass
    
    # Positions
    if pair == 1:
        buy,sell = long_ts1, short_ts1
        
    else:
        buy,sell = long_ts2, short_ts2
    
    return buy,sell
    

# True = Live Trading and False = screener    
live = True


if live:
    current_account_info = mt5.account_info()
    print("------------------------------------------------------------------")
    print("Date: ", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    print(f"Balance: {current_account_info.balance} USD, \t"
          f"Equity: {current_account_info.equity} USD, \t"
          f"Profit: {current_account_info.profit} USD")
    print("------------------------------------------------------------------")


# Initialize the inputs
symbols = ["EURUSD", "EURGBP"]
lots = [0.01, 0.01]
pairs = [1,2]


start = datetime.now().strftime("%H:%M:%S") #"23:59:59"
while True:
    # Verfication for launch
    if datetime.now().weekday() not in (5,6):
        is_time = datetime.now().strftime("%H:%M:%S") == start 
    else:
        is_time = False

    # Launch the algorithm
    if is_time:
        for symbol, lot, pair in zip(symbols, lots, pairs):

            # Create the signals
            buy, sell = pair_trading(symbols[0], symbols[1], pair=pair)

            # Run the algorithm
            if live:
                MT5.run(symbol, buy, sell,lot)

            else:
                print(f"Symbol: {symbol}\t"
                     f"Buy: {buy}\t"
                     f"Sell: {sell}")
    time.sleep(1)

------------------------------------------------------------------
Date:  2021-09-24 07:20:37
Balance: 981.84 USD, 	Equity: 981.84 USD, 	Profit: 0.0 USD
------------------------------------------------------------------
------------------------------------------------------------------
Date:  2021-09-24 07:20:37
SYMBOL: EURUSD
BUY: False 	  SHORT: False
POSITION: None 	 ID: None
------------------------------------------------------------------
------------------------------------------------------------------
Date:  2021-09-24 07:20:37
SYMBOL: EURGBP
BUY: False 	  SHORT: False
POSITION: None 	 ID: None
------------------------------------------------------------------


KeyboardInterrupt: 